# 2.4 Model Selection 소개

## 학습/테스트 데이터 분리 - train_test_split()

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris=load_iris()
dt_clf=DecisionTreeClassifier()
train_data=iris.data
train_label=iris.target
dt_clf.fit(train_data,train_label)

#학습 데이터셋으로 예측수행(잘못된 예시)
pred=dt_clf.predict(train_data)
print('예측정확도',accuracy_score(train_label,pred))

예측정확도 1.0


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf=DecisionTreeClassifier()
iris_data=load_iris()

X_train,X_test,y_train,y_test=train_test_split(iris_data.data,iris_data.target,
                                              test_size=0.3,random_state=121)

In [6]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


넘파이 뿐 아니라 판다스 데이터프레임/series도 train_test_split()으로 분할 가능

In [8]:
import pandas as pd
iris_df=pd.DataFrame(iris_data.data,columns=iris_data.feature_names)
iris_df['target']=iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
ftr_df=iris_df.iloc[:,:-1] #전체 행과 맨끝열 바로 앞까지
tgt_df=iris_df.iloc[:,-1] #전체 행과 맨 끝 열만
X_train,X_test,y_train,y_test=train_test_split(ftr_df,tgt_df,test_size=0.3,random_state=121)

In [10]:
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [11]:
dt_clf = DecisionTreeClassifier( )
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


## 교차 검증

### K-폴드

- 학습-테스트 데이터로 나눈 것 중에, 학습 데이터세트에서 학습폴드/검증 세트로 K번 반복해서 나누기
- 교차검증 최종평가=평균(평가(1~K번까지))
- 학습 데이터 셋 내에서 또 나누는 것
==> 학습데이터 그대로만 학습시켜버리면, 과적합 일어나니까.

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris=load_iris()
features=iris.data
label=iris.target
dt_clf=DecisionTreeClassifier(random_state=156)

#5개 폴드 세트로 분리하는 KFold객체와 
kfold=KFold(n_splits=5)
#폴드 세트별 정확도 담을 리스트 객체 생성
#전체 정확도 계산할것
cv_accuracy=[]
print('붓꽃 데이터 세트 크기:',features.shape[0])

붓꽃 데이터 세트 크기: 150


In [16]:
#반복횟수 변수
n_iter=0

#KFold객체의 split()호출하면 폴드별 학습용, 검증용 테스트의 행 인덱스를 array로 반환
for train_index, test_index in kfold.split(features): #학습안에서 나누니까 features
    #kfold.split()으로 반환된 인덱스 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train,X_test=features[train_index],features[test_index]
    y_train,y_test=label[train_index],label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred=dt_clf.predict(X_test)
    n_iter+=1
    
    #반복 시 마다 정확도 측정
    accuracy=np.round(accuracy_score(y_test,pred),4)
    train_size=X_train.shape[0] #학습 데이터 셋 내의 학습 데이터
    test_size=X_test.shape[0] #학습 데이터 셋 내의 검증 데이터.
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    
    cv_accuracy.append(accuracy)
#개별 iteration별 정확도 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:',np.mean(cv_accuracy))


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


### Stratified K폴드

- 불균형한 분포도를 가진 레이블 데이터 집합 위한 K폴드 방식
- 그냥 전부 stratified k폴드 사용한다고 보면됨
- 학습과 검증 데이터 셋이 가지는 레이블(종속변수 값) 분포도가 유사하도록 검증 데이터 추출

In [22]:
import pandas as pd

iris=load_iris()

iris_df=pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()
iris_df
iris_df['label'].iloc[5]

0

In [23]:
#잘못된 예시
kfold=KFold(n_splits=3)
#kfold.split(X)는 폴드 세트를 3번 반복할때마다 달라지는 학습/테스트 용 데이터 행인덱스 번호 반환
n_iter=0
for train_index, test_index in kfold.split(iris_df):
    n_iter+=1
    #종속변수 값의 학습/검증 분리
    label_train=iris_df['label'].iloc[train_index]
    label_test=iris_df['label'].iloc[test_index]
    
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n',label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

#문제점 : 종속변수 값에 따라서 학습인지 검증인지 정해짐
#(값에 따라 나뉘는게 아니고, 그냥 값은 가만히 있고 개수에따라 나뉘어야됨)

## 교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [24]:
#해결책
#종속변수 값의 분포도가 유사하도록 학습-검증 분리
from sklearn.model_selection import StratifiedKFold

skf=StratifiedKFold(n_splits=3)
n_iter=0

for train_index,test_index in skf.split(iris_df,iris_df['label']):#종속변수(y값)반드시
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

#해결 : 종속변수 값 내부 개수가 달라짐

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64


In [29]:
#skfold로 학습/예측해보기
dt_clf=DecisionTreeClassifier(random_state=156)

skfold=StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

#StratifiedKfold의 split()호출시 반드시 레이블 데이터 셋도 추가 입력 필요
for train_index,test_index in skfold.split(features,label):
    #Split()으로 반환된 인덱스 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test=features[train_index],features[test_index]
    y_train,y_test=label[train_index],label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred=dt_clf.predict(X_test)
    
    #반복 시마다 정확도 측정
    n_iter+=1
    accurac=np.round(accuracy_score(y_test,pred),4)
    train_size=X_train.shape[0]
    test_size=X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :0.7333, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.7333, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.7333, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.7333 0.7333 0.7333]
## 평균 검증 정확도: 0.7333


### cross_val_score()

- 교차검증을 보다 간편하게 하기
- Kfold 클래스 이용한 교차검증 방법 : (폴드 세트 설정>for문으로 학습/예측>최종 성능 평가)
- cross_val_score함수: 폴드세트 추출, 학습/예측, 평가 한번에 수행
- stratified k-fold로 하는것

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris_data=load_iris()
dt_clf=DecisionTreeClassifier(random_state=156)

data=iris_data.data
label=iris_data.target

#성능지표는 정확도(accuracy), 교차검증 세트 3개
scores=cross_val_score(dt_clf, data, label, scoring='accuracy',cv=3)

print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도:',np.round(np.mean(scores),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


## GridSearchCV

- 분류나 회귀와 같은 알고리즘에 사용되는 하이퍼파라미터를 순차적으로 입력하면서 편리하게 '최적의 파라미터' 도출
- 하이퍼파라미터값을 변경하면서 해보기

In [34]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

#데이터 로딩하고 학습데이터와 테스트 데이터 분리
iris=load_iris()
X_train,X_test,y_train,y_test=train_test_split(iris.data, iris.target,test_size=0.2,
                                              random_state=121)
dtree=DecisionTreeClassifier()

#파라미터들을 딕셔너리 형태로 설정
parameters={'max_depth':[1,2,3],'min_samples_split':[2,3]}

In [35]:
import pandas as pd

#param_grid의 하이퍼파라미터들을 3개의 train, test set fold로 나누어서 테스트 수행(cv=3: 3개로 나누고, 교차검증 3번)
#refit=True가 default임. True면 가장 좋은 파라미터 설정으로 재학습시킴
#==> 최적하이퍼파라미터 찾기 위해 6번반복하는데(max_depth*min_samples_split), 3번 교차검증하니까 총 18번학습해서
#   최적 하이퍼파라미터 찾고 그걸로 dtree에 학습 시켜버리는 옵션이 refit
#return_train_score는 없으면 경고떠서 걍 넣은것
grid_dtree=GridSearchCV(dtree,param_grid=parameters,cv=3,refit=True,return_train_score=True)

#붓꽃 train데이터로 param_grid의 하이퍼파라미터들을 순차적으로 학습/평가
grid_dtree.fit(X_train,y_train)# 최적하이퍼파라미터 찾아서 그걸로 학습한것

#GridSearchCV결과는 cv_results_라는 딕셔너리로 저장. 이를 데이터프레임으로 변환하기
scores_df=pd.DataFrame(grid_dtree.cv_results_)#grid_dtree.cv_results_에 저장되는것
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [37]:
print('GridSearchCV 최적 파라미터:',grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

#refit=True로 설정된 GridSearchCV 객체가 fit()수행시 학습이 완료된 Estimator를 내포하고 있으므로 predict()통해 예측도 가능
pred=grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750
테스트 데이터 세트 정확도: 0.9667


In [38]:
#GridSearchCV의 refit으로 이미 학습된 estimator반환
estimator=grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667
